# ENVIRONMENT AND WATER IN THE LOWER MEKONG BASIN


## Table of contents
1. [Mekong Basins](#1)<br>
    1.1 [Hydro basin level 3](#11)<br>
    1.2 [Freshwater Ecoregions](#12)<br>
2. [Mekong Infrastructure Tracker](#2)<br>
    2.1 [Data](#21)<br>
    2.2 [Plotting interative energy map](#22)<br>

5. [References](#5)<br>

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.express as px

# Plot in browser (recommended)
import plotly.io as pio
pio.renderers.default = 'browser'
pd.options.mode.chained_assignment = None

# Make Plotly work in your Jupyter Notebook
# import cufflinks as cf
# import chart_studio.plotly as py
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
# # Use Plotly locally
# cf.go_offline()

In [7]:
from codes.auxiliary import *

## 1. Mekong basins <a id="1"></a>

### 1.1 Hydro basin level 3 <a id="1.1"></a>

This dataset was obtained by delineating drainage basin boundaries from hydrologically corrected elevation data (HydroSHEDS and Hydro1K). Input data resolution is 15 arc-seconds between 60 N and 60 S latitude (based on SRTM), and 30 arc-seconds for higher latitudes (based on GTOPO30). The dataset consists of the following information: numerical code (MAJ_BAS), name (MAJ_NAME) and area (MAJ_AREA) of the major basin in square km.

In [8]:
# Import geodata
hydrobasin_geo = json.load(open("mekong/hydrobasin_l3_mekong.geojson", "r", encoding='utf8'))
hydrobasin_df = pd.read_csv('mekong/hydrobasin_l3_mekong.csv')
hydrobasin_df["AREA_SCALE"] = np.log2(hydrobasin_df['MAJ_AREA'])

# # Convert map properties
region_id_map = convert_id_map(hydrobasin_geo, 'MAJ_NAME', 'id')

#Plotting
fig = px.choropleth_mapbox(
    hydrobasin_df,
    locations = 'id',
    geojson = hydrobasin_geo,
    color = 'AREA_SCALE',
    color_continuous_scale="bluyl",
    range_color=(17, 20),
    hover_name = "MAJ_NAME",
    hover_data = ['SUB_AREA'],
    mapbox_style="carto-positron",
    center={"lat": 18,"lon": 106},
    zoom = 3.5,
    title = "Hydro basins level 3 Mekong",
    )
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

<img src="figures/hydro_basin3_mekong.png" style="width:900px;"/>

### 1.2 Freshwater Ecoregions  <a id="1.2"></a>

This dataset was created from Hydrosheds level 3 dataset to represent approximate freshwater ecoregions of the Mekong basin. Based upon the WWF freshwater ecoregions dataset which is defined as, a large area encompassing one or more freshwater systems that contains a distinct assemblage of natural freshwater communities and species. The freshwater species, dynamics, and environmental conditions within a given ecoregion are more similar to each other than to those of surrounding ecoregions and together form a conservation unit.

In [3]:
# Import data
mekongFreshwater = json.load(open("mekong/mekongfwecoregionshydroshed.geojson", "r"))
Freshwater_df = pd.read_csv('mekong/mekongfwecoregionshydroshed.csv')
Freshwater_df['logarea'] = np.log2(Freshwater_df['sub_area'] )

#Plotting
fig = px.choropleth_mapbox(
    Freshwater_df,
    locations = 'id',
    geojson= mekongFreshwater,
    color = 'logarea',
    color_continuous_scale = "bluyl",
    hover_name = "sub_name",
    hover_data = ['sub_area'],
    mapbox_style = "carto-positron",
    center = {"lat": 18,"lon": 106},
    zoom = 3.5,
    title = "Freshwater Ecoregions of the Mekong Basin",
    )
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

***

## 2 Mekong Infrastructure Tracker <a id="1"></a>

### 2.1 Data <a id="11"></a>

In [4]:
import plotly.graph_objects as go

categories = ['Oil', 'Hydro', 'Waste', 'Coal', 'Biomass', 'Solar', 'Wind', 'Gas', 'Geothermal', 'Mixed Fossil Fuel'] # Order
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey",'purple','yellow','brown','darkgreen','black']
scale = 50

# Import geodata
mekonginfra_geo = json.load(open("mekong/mekong_infrastructure_tracker.geojson", "r", encoding='utf8'))
mekonginfra_df = pd.read_csv('mekong/mekong_infrastructure.csv')

#Edit data
mek_df = mekonginfra_df[['Project Name','Subtype','Year of Completion','Capacity (MW)','Latitude','Longitude']]
mek_df['text'] = mek_df['Project Name'] + '<br> Capacity ' + (mek_df['Capacity (MW)']).astype(str)+'W'
mek_df['Capacity'] = mek_df['Capacity (MW)'].fillna(0)

### 2.2 Plotting interative energy map <a id="22"></a>

Bubble Map with go.Scattergeo

See https://plotly.com/python/reference/scatter/#scatter-marker-sizeref for more information.

Additionally, recommend setting the sizemode attribute: https://plotly.com/python/reference/scatter/#scatter-marker-sizemode to area.

In [5]:
fig = go.Figure()

for i, cat in enumerate(categories):
    mek_sub = mek_df[mek_df['Subtype']==cat]
    fig.add_trace(go.Scattergeo(
        locationmode = 'geojson-id',
        lon = mek_sub['Longitude'],
        lat = mek_sub['Latitude'],
        text = mek_sub['text'],
        marker = dict(
         size = mek_sub['Capacity']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width= 0.3,
            sizemode = 'area'
        ),
        name = '{}'.format(cat)
    )
    )
fig.update_layout(
        title_text = '2020 Mekong Infrastructure Tracker <br>(Click legend to toggle traces)',
        showlegend = True,
        autosize = False,
        width = 1000,  #size of figure
        height = 850,
        geo = go.layout.Geo(
        resolution = 50,
        scope = 'asia',
        showframe = False,
        showcoastlines = True,
        lonaxis_range = [ 90.0, 112.0 ], # cut of asia map
        lataxis_range = [ 5.0, 35.0 ],
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "white" ,
        coastlinecolor = "white",
        projection_type = 'mercator',
        domain = dict(x = [ 0, 1.0 ], y = [ 0, 1.0 ])
    )
)
fig.show()

<img src="figures/mekong_infra.png" style="width:800px;"/>
